In [1]:
from crimson.executable_types_beta.mock import TensorMock
from crimson.executable_types_beta.types import *
from inspect import currentframe
from crimson.executable_types_beta.executor import AnnotationExecutor
from crimson.executable_types_beta.data_model import ExecutionResult
from crimson.executable_types_beta.utils import (
    get_meta_annotations,
    get_meta_annotation,
    generate_meta_dict,
    push_execution_result,
    get_names
)
from crimson.executable_types_beta.test import test_meta_dict
from crimson.types_beta.addon import TypesPack
from crimson.ast_dev_tool import collect_nodes, print_node
import ast


import inspect

def print_object(obj):
    print(inspect.getsource(obj))

In [2]:
class FloatRange(
    float,
    TypesPack[T]
):
	pass

class IntRange(
    float,
    TypesPack[T]
):
	pass

In [3]:
AnnotationExecutor.set_config(
    push_result=True,
    raise_error=False,
)

AnnotationExecutor.env = currentframe().f_locals# {'TensorMock':TensorMock}


In [4]:
array1 = TensorMock.ones((4, 6))
array2 = TensorMock.ones((8, 10))
array3 = TensorMock.ones((4, 8))
array4 = TensorMock.ones((4, 8))

In [5]:
@AnnotationExecutor.with_executable
def func_with_error(
    array1: TensorMock[List, "b", "c"], 
    array2: TensorMock[List, "d == 2*b", "e == b+2*c"],
    array3: TensorMock
) -> TensorMock[List, "b", "f == 3 * b"]:
    return array4

What if you want to push the results, but the `ArgDetails` are too big?

In this case, you'd better customize the `push_execution_result` function.

Check the default `push_execution_result` first.

``` python
def push_execution_result(
    func: FunctionType,
    execution_results: Dict[str, ExecutionResult],
    execution_result: ExecutionResult,
    *args,
    **kwargs
):
    execution_results[func.__name__] = execution_result
```

the `ExecutionResult` includes the arguments passed to the function and the output of the function.

If all of their references are stored in `execution_results`,
they are not collected by the garbage collector.

If we want to analyze or test a DL model, the tensors are too to be stored.
The shape of tensors would be enough.

In [6]:
def convert_tensor_to_shape(arg_details: ArgsDetails_):
	return {key: value.shape for key, value in arg_details.items() if isinstance(value, TensorMock)}

Before

In [ ]:
func_with_error(
    array1,
    array2,
    array3
)

AnnotationExecutor.last_execution_result.args_detail

{'array1': [[1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1]],
 'array2': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'array3': [[1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]],
 'return': [[1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]]}

In [8]:
def my_push_execution_result(
    func,
    execution_results,
    execution_result: ExecutionResult,
    *args,
    **kwargs
):
    execution_result.args_detail = convert_tensor_to_shape(execution_result.args_detail)
    execution_results[func.__name__] = execution_result


AnnotationExecutor.set_functions(
    push_execution_result=my_push_execution_result
)


After

In [9]:
func_with_error(
    array1,
    array2,
    array3
)

AnnotationExecutor.last_execution_result.args_detail

{'array1': (4, 6), 'array2': (8, 10), 'array3': (4, 8), 'return': (4, 8)}

## Meta Dict

You might want to test more than .shape.
In this case, you need to customize the `generate_meta_dict` and `get_meta_annotations`.

I don't recommend to implement it from the skratch.
Let's make a customized one using the pre-defined ones.

In [10]:
@AnnotationExecutor.with_executable
def forward(
        tensor: TensorMock["b", "c", "h", "w"],
        temperature: FloatRange[float, 0, 1],
        depth: IntRange[int, 3, 12],
        extra_arg = 1
    ):
    pass

In [11]:
subscript_nodes = collect_nodes(forward, ast.Subscript)

In [12]:
def get_meta_annotations(
    func: ast.FunctionType,
) -> MetaAnnotations_[Dict[ArgumentName_[str], MetaAnnotation_[List[str] | None]]]:
    meta_annotations = {}
    func_node = collect_nodes(func, ast.FunctionDef)[0]
    arg_nodes = collect_nodes(func, ast.arg)
    for arg_node in arg_nodes:
        arg_name = arg_node.arg
        meta_annotation = get_meta_annotation(arg_node.annotation)

        try:
            if arg_node.annotation.value.id in [FloatRange.__name__, IntRange.__name__]:
                meta_annotation[0] = f"{arg_name} >= {meta_annotation[0]}"
                meta_annotation[1] = f"{arg_name} <= {meta_annotation[1]}"
        except Exception as e:
               e
            

        meta_annotations[arg_node.arg] = meta_annotation

    if hasattr(func_node, "returns"):
        meta_annotations["return"] = get_meta_annotation(func_node.returns)

    return meta_annotations

In [13]:
from typing import NamedTuple, Type

from pydantic import *
from pydantic.fields import FieldInfo


In [14]:
def generate_meta_dict(
    args_details: ArgsDetails_[Dict[str, Any]],
    meta_annotations: MetaAnnotations_[
        Dict[ArgumentName_[str], MetaAnnotation_[List[str]] | None]
    ],
    args_fields:Dict[str, FieldInfo]
) -> MetaDict_[Dict[str, int]]:
	"""
	You will want to implement this by yourself,
	for your customized use.
	"""
	meta_dict = {}
	for name, arg in args_details.items():
		meta_annotation = meta_annotations[name]
		if name != "return":
			arg_type = args_fields[name].annotation
			if meta_annotation is not None:
				names = get_names(meta_annotation=meta_annotation)
				if arg_type.__name__ == TensorMock.__name__:
					for shape_unit, meta_name in zip(arg.shape, names):
						if meta_name not in meta_dict.keys():
							meta_dict[meta_name] = shape_unit
						else:
							if meta_dict[meta_name] != shape_unit:
								raise Exception(
									"Shape Units sharing same name don't have the same value."
								)
				elif arg_type in [FloatRange, IntRange]:
					meta_dict[name] = arg
	return meta_dict

In [15]:
def update_float_range(arg_node: ast.arg, range: List[int]):
    pass

In [16]:
meta_annotations = get_meta_annotations(forward)

In [17]:
meta_annotations

{'tensor': ['b', 'c', 'h', 'w'],
 'temperature': ['temperature >= 0', 'temperature <= 1'],
 'depth': ['depth >= 3', 'depth <= 12'],
 'extra_arg': None,
 'return': None}

In [18]:
AnnotationExecutor.set_functions(
    get_meta_annotations=get_meta_annotations,
    generate_meta_dict=generate_meta_dict
)


In [19]:
tensor = TensorMock.ones((4, 6, 8, 10))

forward(
    tensor = tensor,
    temperature = 0.3,
    depth = 10,   
)

AnnotationExecutor.last_execution_result.test_result

{'temperature >= 0': True,
 'temperature <= 1': True,
 'depth >= 3': True,
 'depth <= 12': True}

In [20]:
forward(
    tensor = tensor,
    temperature = 1.2,
    depth = 20   
)

AnnotationExecutor.last_execution_result.test_result

{'temperature >= 0': True,
 'temperature <= 1': False,
 'depth >= 3': True,
 'depth <= 12': False}